In [1]:
import os
# To disable gpu warnings
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

# Pre-process

In [64]:
def determine_label(l):
    return int(l in {'ok', 'normal', 'moved', 'slightly_moved'})

def instance_reader(file_path):
    def _reader():
        eof = False
        with open(file_path) as f:
            while True:
                while True:
                    _tmp = f.readline()
                    if not _tmp:
                        eof = True
                        break
                    elif _tmp != '\n':
                        break

                if eof:
                    break

                label = None
                instance = np.zeros((15, 6))

                # GHD
                label = determine_label(_tmp.strip())
                
                for i in range(15):
                    content = f.readline()
                    timeline = np.asarray(content[:-1].split('\t')[1:]).astype(np.int64)
                    instance[i] = np.asarray(timeline)

                yield instance, label
    return _reader

In [65]:
data_directory = 'data'
X = list()
y = list()
for datafile in os.listdir(data_directory):
    ins_reader = instance_reader(os.path.join(data_directory, datafile))
    for instance, label in tqdm(ins_reader()):
        X.append(instance)
        y.append(label)

X = np.array(X)
y = np.array(y)

88it [00:00, 5176.77it/s]
47it [00:00, 5542.25it/s]
47it [00:00, 5891.58it/s]
117it [00:00, 6173.22it/s]
164it [00:00, 5667.60it/s]


In [66]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=0.125, random_state=42)

## GHD

In [67]:
xtrain[0][0]

array([12.,  8., 94., -9., 35., -1.])

## Elman Neural Network

In [127]:
class ElmanNeuralNetwork(tf.keras.Model):
    def __init__(self, input_dim, hidden_units, feature_size, n_classes):
        super(ElmanNeuralNetwork, self).__init__()
        self.hidden_units = hidden_units
        self.feature_size = feature_size
        self.input_dim = input_dim
        self.n_classes = n_classes

        self.U = self.add_weight(shape=(self.hidden_units, self.hidden_units), initializer='random_normal', trainable=True)
        self.W = self.add_weight(shape=(self.hidden_units, self.feature_size), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.hidden_units,), initializer='random_normal', trainable=True)

        self.W_y = self.add_weight(shape=(self.n_classes, self.hidden_units), initializer='random_normal', trainable=True)
        self.b_y = self.add_weight(shape=(self.n_classes,), initializer='random_normal', trainable=True)

    def call(self, x):
        state_t = tf.zeros(self.hidden_units)
        for i in range(self.input_dim):
            state_t = tf.keras.activations.tanh(tf.matmul(self.W, tf.reshape(x[0][i], (self.feature_size, 1))) + tf.matmul(self.U, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b, (self.hidden_units, 1)))

        y = tf.keras.activations.tanh(tf.matmul(self.W_y, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b_y, (self.n_classes,1)))
        return y

In [128]:
import datetime
elman_nn = ElmanNeuralNetwork(input_dim=15, hidden_units=10, feature_size=6, n_classes=2)
elman_nn.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                loss='binary_crossentropy',
                metrics=['acc'])
elman_nn.fit(xtrain, ytrain, batch_size=1, epochs=20, callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/datetime-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

Epoch 1/20
323/323 [==============================] - 2s 2ms/step - loss: 2.8131 - acc: 0.6718
Epoch 2/20
323/323 [==============================] - 0s 2ms/step - loss: 2.7272 - acc: 0.7059
Epoch 3/20
323/323 [==============================] - 1s 2ms/step - loss: 2.7147 - acc: 0.7337
Epoch 4/20
323/323 [==============================] - 1s 2ms/step - loss: 2.7020 - acc: 0.7430
Epoch 5/20
323/323 [==============================] - 1s 2ms/step - loss: 2.7019 - acc: 0.7492
Epoch 6/20
323/323 [==============================] - 1s 2ms/step - loss: 2.6944 - acc: 0.7539
Epoch 7/20
323/323 [==============================] - 1s 2ms/step - loss: 2.6926 - acc: 0.7446
Epoch 8/20
323/323 [==============================] - 1s 2ms/step - loss: 2.6874 - acc: 0.7663
Epoch 9/20
323/323 [==============================] - 1s 2ms/step - loss: 2.6793 - acc: 0.7678
Epoch 10/20
323/323 [==============================] - 1s 2ms/step - loss: 2.6770 - acc: 0.7523
Epoch 11/20
323/323 [============================

In [126]:
%load_ext tensorboard

In [ ]:
# tensorflow callback

In [ ]:
# load tensorboard
# tensorboard --logdir=logs/



In [120]:
elman_nn = ElmanNeuralNetwork(input_dim=15, hidden_units=10, feature_size=6, n_classes=2)
elman_nn(xtrain)

tf.Tensor([1], shape=(1,), dtype=int64)


<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.21439831],
       [ 0.23168942]], dtype=float32)>

In [69]:
ytrain

array([0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,